In [ ]:

# LSTM 18 SENTIMENT 
# ----------------------------
# 1. Data Loading and Preprocessing
# ----------------------------

merged_df['close_lag1'] = merged_df['close'].shift(1)
merged_df['volume_lag1'] = merged_df['volume'].shift(1)

# Use sentiment as-is (current day's sentiment, not lagged)
merged_df['sentiment_no_lag'] = merged_df['sentiment_label']

# Drop any rows with NaN values created by shifting
merged_df = merged_df.dropna()


# Split the data into training and testing sets:
# Training: 2018-01-01 to 2019-01-31, Testing: from 2019-02-01 onward
train_df = merged_df[(merged_df['datetime'] >= '2018-01-01') & (merged_df['datetime'] < '2019-02-01')].copy()
test_df = merged_df[merged_df['datetime'] >= '2019-02-01'].copy()

# Initialize scalers for each feature.
# We now have three features: 'close', 'volume', and 'sentiment_label'.
close_scaler = MinMaxScaler()
volume_scaler = MinMaxScaler()
sentiment_scaler = MinMaxScaler()

# Fit and transform on the training set.
train_df['close_scaled'] = close_scaler.fit_transform(train_df[['close']])
train_df['volume_scaled'] = volume_scaler.fit_transform(train_df[['volume']])
train_df['sentiment_scaled'] = sentiment_scaler.fit_transform(train_df[['sentiment_label']])

# Transform the test set using the same scalers.
test_df['close_scaled'] = close_scaler.transform(test_df[['close']])
test_df['volume_scaled'] = volume_scaler.transform(test_df[['volume']])
test_df['sentiment_scaled'] = test_df['sentiment_label']

# Create NumPy arrays combining all three features in the order: [close, volume, sentiment].
train_scaled = train_df[['close_scaled', 'volume_scaled', 'sentiment_scaled']].values
test_scaled  = test_df[['close_scaled', 'volume_scaled', 'sentiment_scaled']].values

# ----------------------------
# 2. Creating Sequences for Time Series
# ----------------------------

lookback = 30  # Number of past days to use as input

def create_sequences(data, lookback):
    X, y = [], []
    for i in range(lookback, len(data)):
        X.append(data[i - lookback:i])
        y.append(data[i, 0])  # The target is the close price (first column)
    return np.array(X), np.array(y)

X_train, y_train = create_sequences(train_scaled, lookback)
X_test, y_test   = create_sequences(test_scaled, lookback)

# ----------------------------
# 3. Model Building and Training
# ----------------------------

model = Sequential()
# Input shape now is (lookback, 3) for our three features.
model.add(LSTM(50, return_sequences=True, input_shape=(lookback, 3)))
model.add(LSTM(50))
model.add(Dense(1))  # Predict a single value: the close price
model.compile(optimizer='adam', loss='mse')

history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.1)

# ----------------------------
# 4. Prediction and Inverse Transformation
# ----------------------------

predictions_scaled = model.predict(X_test)

# Since we only predict the close price, inverse transform using close_scaler.
prediction_with_sentiment = close_scaler.inverse_transform(predictions_scaled)
y_test_actual_with_sentiment = close_scaler.inverse_transform(y_test.reshape(-1, 1))

# Align test dates (first 'lookback' days are lost when creating sequences)
test_dates = test_df['datetime'].values[lookback:]

# ----------------------------
# 5. Visualization: Predicted vs Actual Close Prices
# ----------------------------

plt.figure(figsize=(12, 6))
plt.plot(test_dates, y_test_actual_with_sentiment, label='Actual Close Price', color='blue')
plt.plot(test_dates, prediction_with_sentiment, label='Predicted Close Price', color='red', linestyle='--')
plt.xlabel('Date')
plt.ylabel('BTC Close Price')
plt.title('BTC Close Price Prediction with Discrete Sentiment: Actual vs Predicted')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# ----------------------------
# 6. Performance Metrics
# ----------------------------

mse  = mean_squared_error(y_test_actual_with_sentiment, prediction_with_sentiment)
mae  = mean_absolute_error(y_test_actual_with_sentiment, prediction_with_sentiment)
rmse = np.sqrt(mse)
mape = mean_absolute_percentage_error(y_test_actual_with_sentiment, prediction_with_sentiment) * 100
r2   = r2_score(y_test_actual_with_sentiment, prediction_with_sentiment)

print("Performance Metrics for the LSTM Model (with Discrete Sentiment):")
print(f"MSE: {mse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"MAPE: {mape:.2f}%")
print(f"R2 Score: {r2:.2f}")
